In [166]:
import Pkg

Pkg.add("HiddenMarkovModels")
Pkg.add("Distributions")
Pkg.add("StableRNGs")
Pkg.add("XLSX")
Pkg.add("StatsBase")

   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
   Installed StatsBase ─ v0.34.3
    Updating `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  [2913bbd2] + StatsBase v0.34.3
    Updating `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
  [2913bbd2] ↑ StatsBase v0.34.2 ⇒ v0.34.3
Precomp

In [1]:
using HiddenMarkovModels, Distributions
using LinearAlgebra
using StableRNGs
using XLSX
using DataFrames
using Dates
using Distributions
using Statistics
using CSV
using StatsBase

In [2]:
rng = StableRNG(63);

In [3]:
acwi = DataFrame(XLSX.readtable("../esg/acwi_usd_net_03.04.24.xlsx","acwi_usd_net_full", header=true, infer_eltypes=true))
acwi.Date = Date.(acwi.Date, dateformat"u d, Y")
acwi

Row,Date,ACWI
,Date,Float64
1,1988-01-29,0.0249309
2,1988-02-29,0.0576797
3,1988-03-31,0.0307011
4,1988-04-29,0.0125812
5,1988-05-31,-0.0199201
6,1988-06-30,-0.00119118
7,1988-07-29,0.0183463
8,1988-08-31,-0.0551142
9,1988-09-30,0.0428067


In [5]:
describe(acwi.ACWI)

Summary Stats:
Length:         435
Missing Count:  0
Mean:           0.007221
Std. Deviation: 0.043966
Minimum:        -0.198150
1st Quartile:   -0.017986
Median:         0.012573
3rd Quartile:   0.033632
Maximum:        0.123263
Type:           Float64


In [11]:
init_guess = [0.5, 0.5]
trans_guess = [0.6 0.4; 0.3 0.7]
dists_guess = [MvNormal([-0.4, -0.7], I), MvNormal([0.4, 0.7], I)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess); 

#= init_guess = [0.25, 0.25, 0.25, 0.25]
trans_guess = [0.85 0.05 0.05 0.05; 0.85 0.05 0.05 0.05; 0.85 0.05 0.05 0.05; 0.85 0.05 0.05 0.05]
#dists_guess = [MvNormal([-0.1, -0.7], I), MvNormal([0.4, 0.7], I)]
dists_guess = [Normal(-0.07,0.15),Normal(0.14,0.07), Normal(-0.14,0.2), Normal(0.24,0.01)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess); =#

Hidden Markov Model with:
 - initialization: [0.5, 0.5]
 - transition matrix: [0.6 0.4; 0.3 0.7]
 - observation distributions: [IsoNormal(
dim: 2
μ: [-0.4, -0.7]
Σ: [1.0 0.0; 0.0 1.0]
)
, IsoNormal(
dim: 2
μ: [0.4, 0.7]
Σ: [1.0 0.0; 0.0 1.0]
)
]

In [12]:
hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, acwi.ACWI);

MethodError: MethodError: no method matching logpdf(::IsoNormal, ::Float64)

Closest candidates are:
  logpdf(!Matched::DiscreteUniform, ::Real)
   @ Distributions C:\Users\matsz\.julia\packages\Distributions\UaWBm\src\univariate\discrete\discreteuniform.jl:74
  logpdf(!Matched::Soliton, ::Real)
   @ Distributions C:\Users\matsz\.julia\packages\Distributions\UaWBm\src\univariate\discrete\soliton.jl:98
  logpdf(!Matched::Chernoff, ::Real)
   @ Distributions C:\Users\matsz\.julia\packages\Distributions\UaWBm\src\univariate\continuous\chernoff.jl:155
  ...


In [6]:
last(loglikelihood_evolution)

UndefVarError: UndefVarError: `loglikelihood_evolution` not defined

In [7]:
transition_matrix(hmm_est)

UndefVarError: UndefVarError: `hmm_est` not defined

In [8]:
obs_distributions(hmm_est)

UndefVarError: UndefVarError: `hmm_est` not defined

In [11]:
println("regime 1")
println(mean(obs_distributions(hmm_est)[1])*12)
println(std(obs_distributions(hmm_est)[1])* 12^0.5)
println("regime 2")
println(mean(obs_distributions(hmm_est)[2])*12)
println(std(obs_distributions(hmm_est)[2])* 12^0.5)

regime 1
0.019032423436001054
0.190628291589636
regime 2
0.16163287752143957
0.08729078654259356


In [462]:
acwi_short = acwi[Date(2003,01,01) .<= acwi.Date .<= Date(2024,03,31), :]

Row,Date,ACWI
,Date,Float64
1,2003-01-31,-0.0294357
2,2003-02-28,-0.017938
3,2003-03-31,-0.00431422
4,2003-04-30,0.0886378
5,2003-05-30,0.0574971
6,2003-06-30,0.0187239
7,2003-07-31,0.0218749
8,2003-08-29,0.0233834
9,2003-09-30,0.00607447


In [13]:
println(mean(acwi.ACWI))
println(std(acwi.ACWI))
println(mean(acwi_short.ACWI))
println(std(acwi_short.ACWI))


0.007221256079983577
0.04396581164211372
0.00820031798641316
0.044712591384382244


In [14]:
hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, acwi_short.ACWI);

In [15]:
last(loglikelihood_evolution)

459.13892457264535

In [16]:
typeof(hmm_est)

HMM{Vector{Float64}, Matrix{Float64}, Vector{Normal{Float64}}}

In [17]:
transition_matrix(hmm_est)

2×2 Matrix{Float64}:
 0.932483   0.0675174
 0.0569406  0.943059

In [18]:
obs_distributions(hmm_est)

2-element Vector{Normal{Float64}}:
 Normal{Float64}(μ=0.001677850881617401, σ=0.05909238647054062)
 Normal{Float64}(μ=0.014143605791660168, σ=0.02345202862629552)

In [19]:
println("regime 1")
println(mean(obs_distributions(hmm_est)[1])*12)
println(std(obs_distributions(hmm_est)[1])* 12^0.5)
println("regime 2")
println(mean(obs_distributions(hmm_est)[2])*12)
println(std(obs_distributions(hmm_est)[2])* 12^0.5)

regime 1
0.020134210579408812
0.20470203141494414
regime 2
0.169723269499922
0.08124021024260716


In [20]:
initialization(hmm_est)

2-element Vector{Float64}:
 1.0
 5.644524055265908e-50

In [42]:
cpipl = CSV.read("../esg/cpiypl_m.csv", DataFrame)[:,["Data","Zamkniecie"]]

Row,Data,Zamkniecie
,Date,Float64
1,2002-12-31,0.8
2,2003-01-31,0.5
3,2003-02-28,0.5
4,2003-03-31,0.6
5,2003-04-30,0.3
6,2003-05-31,0.4
7,2003-06-30,0.8
8,2003-07-31,0.8
9,2003-08-31,0.7


In [43]:
date_index = Date(2003,01,01) .<= cpipl.Data .<= Date(2024,02,29)
edo = cpipl[date_index,"Zamkniecie"] ./100 .+ 0.0125

254-element Vector{Float64}:
 0.0175
 0.0175
 0.018500000000000003
 0.0155
 0.0165
 0.0205
 0.0205
 0.0195
 0.021500000000000002
 0.025500000000000002
 ⋮
 0.1275
 0.12050000000000001
 0.11349999999999999
 0.09449999999999999
 0.0785
 0.0785
 0.0745
 0.051500000000000004
 0.040499999999999994

In [44]:
acwi_s = acwi[Date(2003,01,01) .<= acwi.Date .<= Date(2024,02,29),"ACWI"]

254-element Vector{Float64}:
 -0.029435692626285204
 -0.01793797150041898
 -0.0043142245259009115
  0.08863778054862848
  0.05749710080604764
  0.018723853620892683
  0.02187491693910637
  0.023383446912552674
  0.006074469437031604
  0.06037793047696027
  ⋮
  0.05806157327945405
  0.03660848514326154
 -0.027944324625502892
 -0.04134978511934306
 -0.030067745405442547
  0.09229752768158961
  0.04803335439585643
  0.0058616798157242744
  0.042913606684614525

In [45]:
returns = hcat(acwi_s,edo)

254×2 Matrix{Float64}:
 -0.0294357   0.0175
 -0.017938    0.0175
 -0.00431422  0.0185
  0.0886378   0.0155
  0.0574971   0.0165
  0.0187239   0.0205
  0.0218749   0.0205
  0.0233834   0.0195
  0.00607447  0.0215
  0.0603779   0.0255
  ⋮           
  0.0580616   0.1275
  0.0366085   0.1205
 -0.0279443   0.1135
 -0.0413498   0.0945
 -0.0300677   0.0785
  0.0922975   0.0785
  0.0480334   0.0745
  0.00586168  0.0515
  0.0429136   0.0405

In [46]:
returns = [returns[i,:] for i in 1:size(returns,1)]

254-element Vector{Vector{Float64}}:
 [-0.029435692626285204, 0.0175]
 [-0.01793797150041898, 0.0175]
 [-0.0043142245259009115, 0.018500000000000003]
 [0.08863778054862848, 0.0155]
 [0.05749710080604764, 0.0165]
 [0.018723853620892683, 0.0205]
 [0.02187491693910637, 0.0205]
 [0.023383446912552674, 0.0195]
 [0.006074469437031604, 0.021500000000000002]
 [0.06037793047696027, 0.025500000000000002]
 ⋮
 [0.05806157327945405, 0.1275]
 [0.03660848514326154, 0.12050000000000001]
 [-0.027944324625502892, 0.11349999999999999]
 [-0.04134978511934306, 0.09449999999999999]
 [-0.030067745405442547, 0.0785]
 [0.09229752768158961, 0.0785]
 [0.04803335439585643, 0.0745]
 [0.0058616798157242744, 0.051500000000000004]
 [0.042913606684614525, 0.040499999999999994]

In [93]:
init_guess = [0.25, 0.25, 0.25, 0.25]
trans_guess = [0.25 0.25 0.25 0.25; 0.25 0.25 0.25 0.25; 0.25 0.25 0.25 0.25; 0.25 0.25 0.25 0.25]
dists_guess = [MvNormal([-0.05, 0.012], I), MvNormal([0.2, 0.037], I), MvNormal([0.07, 0.049], I), MvNormal([-0.2, 0.082], I)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess);

In [94]:
hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, returns);

In [95]:
last(loglikelihood_evolution)

1086.0423474913564

In [96]:
transition_matrix(hmm_est)

4×4 Matrix{Float64}:
 0.425494     0.234221     0.283825     0.0564599
 0.187719     0.812281     7.55407e-13  3.11054e-20
 0.217072     0.644388     0.13854      5.46727e-28
 4.81994e-21  8.57241e-37  0.0942793    0.905721

In [396]:
acwi_regimes = obs_distributions(hmm_est)

4-element Vector{IsoNormal}:
 IsoNormal(
dim: 2
μ: [-0.03129398017170018, 0.03477600738241481]
Σ: [0.0005866814416253256 0.0; 0.0 0.0005866814416253256]
)

 IsoNormal(
dim: 2
μ: [0.019660866124438896, 0.032555661006222585]
Σ: [0.00030808785285821175 0.0; 0.0 0.00030808785285821175]
)

 IsoNormal(
dim: 2
μ: [0.07926253831291416, 0.037884960896446176]
Σ: [0.0004508494595335847 0.0; 0.0 0.0004508494595335847]
)

 IsoNormal(
dim: 2
μ: [-0.021833161896736286, 0.1185034538258361]
Σ: [0.003512190550272683 0.0; 0.0 0.003512190550272683]
)


In [402]:
mean(acwi_regimes[1])

2-element Vector{Float64}:
 -0.03129398017170018
  0.03477600738241481

In [400]:
println("$(mean(acwi_regimes[1])), $(std(acwi_regimes[1]))")


MethodError: MethodError: no method matching iterate(::IsoNormal)

Closest candidates are:
  iterate(!Matched::Base.AsyncGenerator, !Matched::Base.AsyncGeneratorState)
   @ Base asyncmap.jl:362
  iterate(!Matched::Base.AsyncGenerator)
   @ Base asyncmap.jl:362
  iterate(!Matched::RegexMatch, Any...)
   @ Base regex.jl:284
  ...


In [52]:
regime1 = obs_distributions(hmm_est)[1]

IsoNormal(
dim: 2
μ: [-0.007180069470621233, 0.07578098201327435]
Σ: [0.0036875320284281313 0.0; 0.0 0.0036875320284281313]
)


In [53]:
mean(regime1) * [12. 1.]

2×2 Matrix{Float64}:
 -0.0861608  -0.00718007
  0.909372    0.075781

In [54]:
cov(regime1)

2×2 Matrix{Float64}:
 0.00368753  0.0
 0.0         0.00368753

In [55]:
cov(regime1)[1,1]*12^0.5

0.01277398565555009

In [56]:
init = [0.6, 0.4]
trans = [0.7 0.3; 0.2 0.8]
dists = [MvNormal([-0.5, -0.8], I), MvNormal([0.5, 0.8], I)]
hmm = HMM(init, trans, dists)

Hidden Markov Model with:
 - initialization: [0.6, 0.4]
 - transition matrix: [0.7 0.3; 0.2 0.8]
 - observation distributions: [IsoNormal(
dim: 2
μ: [-0.5, -0.8]
Σ: [1.0 0.0; 0.0 1.0]
)
, IsoNormal(
dim: 2
μ: [0.5, 0.8]
Σ: [1.0 0.0; 0.0 1.0]
)
]

In [65]:
nb_seqs = 100
long_obs_seqs = [last(rand(rng, hmm, 120)) for k in 1:nb_seqs];
typeof(long_obs_seqs)

Vector{Vector{Vector{Float64}}} (alias for Array{Array{Array{Float64, 1}, 1}, 1})

In [66]:
long_obs_seq_concat = reduce(vcat, long_obs_seqs)
typeof(long_obs_seq_concat)

Vector{Vector{Float64}} (alias for Array{Array{Float64, 1}, 1})

In [67]:
seq_ends = cumsum(length.(long_obs_seqs))
seq_ends'

1×100 adjoint(::Vector{Int64}) with eltype Int64:
 120  240  360  480  600  720  840  960  …  11520  11640  11760  11880  12000

In [68]:
init_guess = [0.5, 0.5]
trans_guess = [0.6 0.4; 0.3 0.7]
dists_guess = [MvNormal([-0.4, -0.7], I), MvNormal([0.4, 0.7], I)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess);

In [75]:
hmm_est_concat, _ = baum_welch(hmm_guess, long_obs_seq_concat,; seq_ends);

In [76]:
transition_matrix(hmm_est_concat)

2×2 Matrix{Float64}:
 0.686014  0.313986
 0.1935    0.8065

In [77]:
obs_distributions(hmm_est)

2-element Vector{IsoNormal}:
 IsoNormal(
dim: 2
μ: [-0.007180069470621233, 0.07578098201327435]
Σ: [0.0036875320284281313 0.0; 0.0 0.0036875320284281313]
)

 IsoNormal(
dim: 2
μ: [0.014715723918157661, 0.03150717749765352]
Σ: [0.00048574567964616865 0.0; 0.0 0.00048574567964616865]
)


#### HMM on scenarios

In [328]:
simulated_ACWI = Matrix(CSV.read("../sb_gbi/equity_scenarios.csv",DataFrame))[1:1000,:]
simulated_ACWI = reshape(transpose(simulated_ACWI),(50000,1))

50000×1 reshape(transpose(::Matrix{Float64}), 50000, 1) with eltype Float64:
  0.183608
  0.040727
  0.130638
  0.167958
 -0.178384
  0.032183
  0.122386
 -0.049013
 -0.230813
 -0.230181
  ⋮
 -0.167158
 -0.16384
  0.079
  0.037215
 -0.039934
 -0.097223
  0.129483
  0.165093
  0.100469

In [327]:
inflation = Matrix(CSV.read("../sb_gbi/inflation_scenarios.csv", DataFrame))[1:1000,2:end]
edo= reshape(transpose(inflation),(50000,1))

50000×1 reshape(transpose(::Matrix{Float64}), 50000, 1) with eltype Float64:
 0.035158
 0.031416
 0.046096
 0.018379
 0.043002
 0.029849
 0.043892
 0.033638
 0.027528
 0.079481
 ⋮
 0.03732
 0.083623
 0.079181
 0.114893
 0.110842
 0.077726
 0.048074
 0.026433
 0.033942

In [331]:
returns = cat(simulated_ACWI,edo,dims=2)

50000×2 Matrix{Float64}:
  0.183608  0.035158
  0.040727  0.031416
  0.130638  0.046096
  0.167958  0.018379
 -0.178384  0.043002
  0.032183  0.029849
  0.122386  0.043892
 -0.049013  0.033638
 -0.230813  0.027528
 -0.230181  0.079481
  ⋮         
 -0.167158  0.03732
 -0.16384   0.083623
  0.079     0.079181
  0.037215  0.114893
 -0.039934  0.110842
 -0.097223  0.077726
  0.129483  0.048074
  0.165093  0.026433
  0.100469  0.033942

In [334]:
returns = [returns[i,:] for i in 1:size(returns,1)]

50000-element Vector{Vector{Float64}}:
 [0.183608, 0.035158]
 [0.040727, 0.031416]
 [0.130638, 0.046096]
 [0.167958, 0.018379]
 [-0.178384, 0.043002]
 [0.032183, 0.029849]
 [0.122386, 0.043892]
 [-0.049013, 0.033638]
 [-0.230813, 0.027528]
 [-0.230181, 0.079481]
 ⋮
 [-0.167158, 0.03732]
 [-0.16384, 0.083623]
 [0.079, 0.079181]
 [0.037215, 0.114893]
 [-0.039934, 0.110842]
 [-0.097223, 0.077726]
 [0.129483, 0.048074]
 [0.165093, 0.026433]
 [0.100469, 0.033942]

In [345]:
n_scenarios = 8

8

In [336]:
function guess_init(n_scenarios)
    regimes_probs = rand(1:100,n_scenarios)
    return regimes_probs/sum(regimes_probs)
end

function guess_tmatrix(n_scenarios)
    regimes_probs = rand(1:100,n_scenarios,n_scenarios)
    return regimes_probs ./ sum(regimes_probs,dims=2)
end

guess_tmatrix(4)

4×4 Matrix{Float64}:
 0.115183  0.387435  0.235602   0.26178
 0.215827  0.28777   0.151079   0.345324
 0.178771  0.435754  0.0614525  0.324022
 0.176056  0.309859  0.204225   0.309859

In [309]:
x = [1 2; 3 4]
sum(x,dims=1)

1×2 Matrix{Int64}:
 4  6

In [271]:
function guess_distribution()
    corr = Matrix(1.0I,2,2)
    sigma = [rand(Uniform(0,0.25)),rand(Uniform(0,0.035))]
    means = [rand(Uniform(-0.5,0.5)),rand(Uniform(-0.01,0.12))]
    cov = cor2cov(corr,sigma)
    return MvNormal(means,cov)
end

guess_distribution()

FullNormal(
dim: 2
μ: [-0.4682975640736762, 0.10606126486839945]
Σ: [0.004287247891805659 0.0; 0.0 0.0006856618250164061]
)


In [302]:
guess_matrix = [guess_tmatrix(n_scenarios) for s in 1:n_scenarios][:,:]

4×1 Matrix{Matrix{Float64}}:
 [0.11347517730496454; 0.524822695035461; 0.2198581560283688; 0.14184397163120568;;]
 [0.46368715083798884; 0.07262569832402235; 0.2905027932960894; 0.17318435754189945;;]
 [0.2052401746724891; 0.4104803493449782; 0.33624454148471616; 0.048034934497816595;;]
 [0.3357664233576642; 0.2773722627737226; 0.3613138686131387; 0.025547445255474453;;]

In [413]:
n_scenarios = 8

8

In [414]:
init_guess = guess_init(n_scenarios)
guess_matrix = guess_tmatrix(n_scenarios)
guess_dist = [guess_distribution() for s in 1:n_scenarios]
hmm_guess = HMM(init_guess, guess_matrix, guess_dist);
println(hmm_guess)

Hidden Markov Model with:
 - initialization: [0.12813370473537605, 0.08913649025069638, 0.21727019498607242, 0.11977715877437325, 0.14484679665738162, 0.10863509749303621, 0.011142061281337047, 0.181058495821727]
 - transition matrix: [0.17429193899782136 0.1830065359477124 0.1721132897603486 0.11982570806100218 0.11328976034858387 0.15904139433551198 0.05010893246187364 0.02832244008714597; 0.22350230414746544 0.013824884792626729 0.06912442396313365 0.1336405529953917 0.08064516129032258 0.10599078341013825 0.14285714285714285 0.2304147465437788; 0.13164556962025317 0.1949367088607595 0.11139240506329114 0.10126582278481013 0.043037974683544304 0.1670886075949367 0.020253164556962026 0.23037974683544304; 0.17207792207792208 0.25 0.17532467532467533 0.17857142857142858 0.048701298701298704 0.1266233766233766 0.01948051948051948 0.02922077922077922; 0.10677083333333333 0.10416666666666667 0.19010416666666666 0.046875 0.18489583333333334 0.0546875 0.203125 0.109375; 0.16955684007707128 

In [415]:
hmm_est_concat, _ = baum_welch(hmm_guess, returns);

In [416]:
transition_matrix(hmm_est_concat)

8×8 Matrix{Float64}:
 0.124894    0.0778872  0.364417   …  0.0012784    0.00396535  1.99943e-5
 0.00517672  0.210788   0.241252      0.00110419   0.213919    0.00141429
 0.00607577  0.262033   0.338491      0.00636439   0.0719698   0.0147117
 0.0025987   0.272332   0.19364       0.000329305  0.0897576   0.0010582
 0.00661068  0.0485615  0.241535      0.0263314    0.231744    0.07482
 2.33756e-5  0.0274464  0.146455   …  0.131038     0.173441    0.121758
 8.22636e-5  0.116186   0.165916      0.0340987    0.449113    0.0174527
 8.52012e-7  0.0371716  0.0768737     0.052661     0.218088    0.394288

In [417]:
obs_distributions(hmm_est_concat)

8-element Vector{FullNormal}:
 FullNormal(
dim: 2
μ: [-0.2870289938238464, 0.04038257697366846]
Σ: [0.0015098833411718228 -0.00023224108192998577; -0.00023224108192998577 0.00027245047146650324]
)

 FullNormal(
dim: 2
μ: [0.09743041957177664, 0.023627291841069813]
Σ: [0.02264085374741083 4.011535373175166e-5; 4.011535373175166e-5 4.888966658536926e-5]
)

 FullNormal(
dim: 2
μ: [0.05075159098217517, 0.03944159751197723]
Σ: [0.027882740736259612 9.466614628477535e-5; 9.466614628477535e-5 0.0001415836773600475]
)

 FullNormal(
dim: 2
μ: [0.08085442741766322, 0.012299999999998711]
Σ: [0.02503255696613832 4.1535086663141225e-31; 4.1535086663141225e-31 1.6612620310453574e-30]
)

 FullNormal(
dim: 2
μ: [0.007180478306713866, 0.06568102679492421]
Σ: [0.025270089225719387 0.00042801217240780106; 0.00042801217240780106 0.00020258906151481368]
)

 FullNormal(
dim: 2
μ: [0.021842174635699554, 0.08832061282552156]
Σ: [0.026487472671298487 7.631210798899707e-5; 7.631210798899707e-5 0.000112894506243

In [487]:
dt = sqrt(1/12)

0.28867513459481287

In [489]:
n_scenarios = 4

init_guess = guess_init(n_scenarios)
guess_matrix = guess_tmatrix(n_scenarios)
guess_dist = [Normal(-0.25/12,0.2*dt),Normal(0.0/12,0.1*dt),Normal(0.2/12,0.1*dt),Normal(0.29/12,0.2*dt)]
hmm_guess = HMM(init_guess, guess_matrix, guess_dist);
println(hmm_guess)

Hidden Markov Model with:
 - initialization: [0.3509933774834437, 0.08609271523178808, 0.18543046357615894, 0.37748344370860926]
 - transition matrix: [0.43636363636363634 0.3151515151515151 0.13333333333333333 0.11515151515151516; 0.379746835443038 0.21518987341772153 0.3628691983122363 0.04219409282700422; 0.10179640718562874 0.1497005988023952 0.32335329341317365 0.4251497005988024; 0.23684210526315788 0.15789473684210525 0.5789473684210527 0.02631578947368421]
 - observation distributions: [Normal{Float64}(μ=-0.020833333333333332, σ=0.057735026918962574), Normal{Float64}(μ=0.0, σ=0.028867513459481287), Normal{Float64}(μ=0.016666666666666666, σ=0.028867513459481287), Normal{Float64}(μ=0.024166666666666666, σ=0.057735026918962574)]


In [490]:
hmm_est_acwi, _ = baum_welch(hmm_guess, acwi.ACWI);

In [491]:
transition_matrix(hmm_est_acwi)

4×4 Matrix{Float64}:
 0.923315    2.13206e-5  0.0766638  1.29378e-9
 0.424579    0.0022659   0.573134   2.1781e-5
 6.8342e-6   0.0115333   0.491766   0.496694
 0.00163101  0.386883    0.610195   0.00129055

In [492]:
mean.(obs_distributions(hmm_est_acwi)) .* 12

4-element Vector{Float64}:
  0.001041078203392006
 -0.10824173729011502
  0.3335081424409114
 -0.13867385512961183

In [493]:
std.(obs_distributions(hmm_est_acwi)) .* sqrt(12)

4-element Vector{Float64}:
 0.20674763970848187
 0.05621377894130931
 0.0647863067897706
 0.09070018137218788